In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
import xarray as xr
import os 
import shutil

p = Path(".")
root_folder = p.cwd().parent
root_folder
data_folder = root_folder / 'pjm_2023_test' / 'pjm_data'
data_folder
scripts_foler = root_folder/ 'pjm_2023_test'

In [2]:
#Replace with file path for this directory on your machine
!pip install -e /Users/justinmaynard/Documents/GitHub/assetraMP/ 


Obtaining file:///Users/justinmaynard/Documents/GitHub/assetraMP
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for assetra (pyproject.toml) ... done
  Created wheel for assetra: filename=assetra-1.0.3-py3-none-any.whl size=2689 sha256=dbec1acd386afa31ce06d71424f177f8781dc8432d27f26b9e6f1bc595d1c07a
  Stored in directory: /private/var/folders/zd/3zqsbnbn5lg29wjyl0df33l80000gn/T/pip-ephem-wheel-cache-00l4a8r4/wheels/c0/f5/c9/3b09bf7f2c6beadc5d6cde6ca7e50094804a4b90493c211e80
Successfully built assetra
  Attempting uninstall: assetra
    Found existing installation: assetra 1.0.3
    Uninstalling assetra-1.0.3:
      Successfully uninstalled assetra-1.0.3


In [3]:
#Keep outside function
#Function to load demand data
def load_pjm_cleaned_hourly_demand(
        pjm_demand_file: Path,
        start_hour: datetime,
        end_hour: datetime) -> xr.DataArray:
    """Return hourly demand data as formatted data array.
    To use this function, download cleaned demand data from:

    https://github.com/truggles/EIA_Cleaned_Hourly_Electricity_Demand_Data

    Args:
        eia_930_cleaned_demand_file (Path): Path to hourly demand file
        start_hour (datetime): First timestamp to include
        end_hour (datetime): Last timestamp to include (inclusive)

    Returns:
        xr.DataArray: Hourly demand array with time dimension and datetime coordinates.
    """
    # read demand file
    pjm_demand_df = pd.read_csv(
        pjm_demand_file,
        usecols=["datetime_beginning_ept", "mw"],
        index_col="datetime_beginning_ept",
        parse_dates=True,
    )

    # keep cleaned demand demand
    pjm_hourly_demand_pd = pjm_demand_df["mw"].loc[start_hour:end_hour]

    # convert to xr.DataArray
    pjm_hourly_demand = xr.DataArray(
        data=pjm_hourly_demand_pd.values,
        coords=dict(
            time=pjm_hourly_demand_pd.index.values
        )
    )
    return pjm_hourly_demand

In [4]:
demand_path = Path(data_folder / "pjm_load_summed_cleaned_updated.csv")
power_generation_files = [
    Path(data_folder / "pjm_power_generation_2022.nc"),
    Path(data_folder / "pjm_power_generation_2023.nc")
]
power_generation_file_combined = xr.concat([xr.open_dataset(file) for file in power_generation_files], dim='time')
start_hour = "2022-01-01 00:00:00"
end_hour = "2023-12-31 23:00:00"
plant_file = data_folder / "eia_860_plants.csv"
thermal_file = data_folder / "eia_860_thermal_generators.csv"
wind_file = data_folder / "eia_860_wind_generators.csv"
solar_file = data_folder / "eia_860_solar_generators.csv"
storage_file = data_folder / "eia_860_storage_generators.csv"


eia_860_plants = Path(data_folder / "eia_860_plants.csv")
eia_860_thermal_generators = Path(data_folder / "eia_860_thermal_generators.csv")
eia_860_wind_generators = Path(data_folder / "eia_860_wind_generators.csv")
eia_860_solar_generators = Path(data_folder / "eia_860_solar_generators.csv")
eia_860_storage_generators = Path(data_folder / "eia_860_storage_generators.csv")

def load_energy_system(demand_file, start_hour, end_hour,
                        plant_file, thermal_file, solar_file, wind_file, storage_file, pow_gen_dataset, class_to_test_input):
    
    #load plants and generators
    eia_860_plants = pd.read_csv(plant_file, index_col=0)
    eia_860_thermal_generators = pd.read_csv(thermal_file, index_col=0)
    eia_860_wind_generators = pd.read_csv(wind_file, index_col=0)
    eia_860_solar_generators = pd.read_csv(solar_file, index_col=0)
    eia_860_storage_generators = pd.read_csv(storage_file, index_col=0)
    eia_860_thermal_generators = eia_860_thermal_generators.dropna(subset=['MC'])

    #Load demand data
    hourly_demand = load_pjm_cleaned_hourly_demand(
        demand_file,
        start_hour="2022-01-01 00:00:00",
        end_hour="2023-12-31 23:00:00"
        )

    from assetra.system import EnergySystem
    from assetra.system import EnergySystemBuilder

    builder = EnergySystemBuilder()
    unit_count = 0

    # create demand unit
    from assetra.units import DemandUnit

    builder.add_unit(
        DemandUnit(
            id=unit_count,
            hourly_demand=hourly_demand
        )
    )
    unit_count += 1


    def get_nearest_hourly_profile(
        latitude: float,
        longitude: float,
        array: xr.DataArray
    ) -> xr.DataArray:
        """Return time series corresponding to the nearest coordinate in a
        MERRA power generation data array.

        Args:
            latitude (float): Latitude relative to equator in degrees
            start_hour (datetime): Longitude relative to meridian in degrees
            array (xr.DataArray): "solar_capacity_factor", "wind_capacity_factor",
                or "temperature"

        Returns:
            xr.DataArray: Array with time dimension and datetime coordinates.
        """
        return array.sel(
                lat=latitude, 
                lon=longitude, 
                method="nearest"
            ).squeeze(drop=True)

    def get_merra_power_generation_solar_cf(
        latitude: float,
        longitude: float) -> xr.DataArray:
        return get_nearest_hourly_profile(latitude, longitude, pow_gen_dataset["solar_capacity_factor"])

    def get_merra_power_generation_wind_cf(
        latitude: float,
        longitude: float) -> xr.DataArray:
        return get_nearest_hourly_profile(latitude, longitude, pow_gen_dataset["wind_capacity_factor"])

    def get_merra_power_generation_temperature(
        latitude: float,
        longitude: float) -> xr.DataArray:
        return get_nearest_hourly_profile(latitude, longitude, pow_gen_dataset["temperature"])
    
    

    # load temperature dependent outage rate (tdfor) table
    tdfor_table_file = Path(data_folder / "temperature_dependent_outage_rates.csv")
    tdfor_table = pd.read_csv(tdfor_table_file, index_col=0)
    tdfor_table = tdfor_table / 100 # percentages stored as integers

    # create mapping table for tdfor table
    tech_categories = {
        "CC" : ["Natural Gas Fired Combined Cycle"],
        "CT" : ["Natural Gas Fired Combustion Turbine","Landfill Gas"],
        "DS" : ["Natural Gas Internal Combustion Engine"],
        "ST" : ["Conventional Steam Coal","Natural Gas Steam Turbine"],
        "NU" : ["Nuclear"],
        "HD" : ["Conventional Hydroelectric","Solar Thermal without Energy Storage",
                    "Hydroelectric Pumped Storage","Solar Thermal with Energy Storage","Wood/Wood Waste Biomass"]
    }

    # create mapping from technology to category
    tech_mapping = {tech : cat for cat, techs in tech_categories.items() for tech in techs}

    def get_hourly_forced_outage_rate(hourly_temperature: xr.DataArray, technology: str) -> xr.DataArray:
        # index tdfor table by tech
        tdfor_map = tdfor_table[tech_mapping.get(technology, "Other")]
        map_temp_to_for = lambda hourly_temperature: tdfor_map.iloc[
                tdfor_map.index.get_indexer(hourly_temperature, method="nearest")
            ]
        return xr.apply_ufunc(
            map_temp_to_for,
            hourly_temperature
        ).rename("hourly_forced_outage_rate")
    
    from assetra.units import StochasticUnit

    for _, generator in eia_860_thermal_generators.iterrows():
        # get hourly temperature
        hourly_temperature = get_merra_power_generation_temperature(
            generator["Latitude"],
            generator["Longitude"]
        )

        # map temperature to hourly forced outage rate
        hourly_forced_outage_rate = get_hourly_forced_outage_rate(hourly_temperature, generator["Technology"])

        # get hourly capacity
        hourly_capacity = ( 
            xr.ones_like(hourly_temperature).rename("hourly_capacity") 
            * generator["Nameplate Capacity (MW)"]
        )

        # create assetra energy unit
        thermal_unit = StochasticUnit(
                id=unit_count,
                nameplate_capacity=generator["Nameplate Capacity (MW)"],
                hourly_capacity=hourly_capacity,
                hourly_forced_outage_rate=hourly_forced_outage_rate,
                marginal_cost = generator["MC"],
            )
        unit_count += 1
        
        # add unit to energy system
        builder.add_unit(thermal_unit)


    # add solar 
    for _, generator in eia_860_solar_generators.iterrows():
        # get hourly temperature
        hourly_temperature = get_merra_power_generation_temperature(
            generator["Latitude"],
            generator["Longitude"]
        )
        # get hourly temperature
        hourly_capacity = get_merra_power_generation_solar_cf(
            generator["Latitude"],
            generator["Longitude"]
        ) * generator["Nameplate Capacity (MW)"]

        # map temperature to hourly forced outage rate
        hourly_forced_outage_rate = get_hourly_forced_outage_rate(hourly_temperature, generator["Technology"])

        # create assetra energy unit
        solar_unit = StochasticUnit(
                id=unit_count,
                nameplate_capacity=generator["Nameplate Capacity (MW)"],
                hourly_capacity=hourly_capacity,
                hourly_forced_outage_rate=hourly_forced_outage_rate,
                marginal_cost = generator["MC"], #0,
                )
        unit_count += 1
        
        # add unit to energy system
        builder.add_unit(solar_unit)

    # add wind
    for _, generator in eia_860_wind_generators.iterrows():
        # get hourly temperature
        hourly_temperature = get_merra_power_generation_temperature(
            
            generator["Latitude"],
            generator["Longitude"]
        )
        # get hourly temperature
        hourly_capacity = get_merra_power_generation_wind_cf(
            generator["Latitude"],
            generator["Longitude"]
        ) * generator["Nameplate Capacity (MW)"]

        # map temperature to hourly forced outage rate
        hourly_forced_outage_rate = get_hourly_forced_outage_rate(hourly_temperature, generator["Technology"])


        # create assetra energy unit
        wind_unit = StochasticUnit(
                id=unit_count,
                nameplate_capacity=generator["Nameplate Capacity (MW)"],
                hourly_capacity=hourly_capacity,
                hourly_forced_outage_rate=hourly_forced_outage_rate,
                marginal_cost = generator["MC"],# 0,
                )
        unit_count += 1
        
        # add unit to energy system
        builder.add_unit(wind_unit)


    #add storage
    from assetra.units import StorageUnit
    STORAGE_EFFICIENCY = 0.85

    class_to_test = class_to_test_input
    class_to_test_lst = []


    for _, generator in eia_860_storage_generators.iterrows():
        storage_duration = generator["Nameplate Energy Capacity (MWh)"] / generator["Nameplate Capacity (MW)"]
        if storage_duration < 4:
            storage_class = '2'
        elif 4 <= storage_duration < 6:
            storage_class = '4'
        elif 6 <= storage_duration < 8:
            storage_class = '6'
        elif 8 <= storage_duration < 10:
            storage_class = '8'
        else:
            storage_class = '12'

        if storage_class == class_to_test:
            class_to_test_lst.append(generator)
        else:
            storage_unit = StorageUnit(
                id=unit_count,
                nameplate_capacity=generator["Nameplate Capacity (MW)"],
                charge_rate=generator["Nameplate Capacity (MW)"],
                discharge_rate=generator["Nameplate Capacity (MW)"],
                charge_capacity=generator["Nameplate Energy Capacity (MWh)"],
                roundtrip_efficiency = STORAGE_EFFICIENCY,
                storage_duration = generator["Nameplate Energy Capacity (MWh)"] / generator["Nameplate Capacity (MW)"],
                storage_class = storage_class       

                
            )
            unit_count += 1

            # add unit to energy system
            builder.add_unit(storage_unit)
        class_to_test_df = pd.DataFrame(class_to_test_lst, columns=eia_860_storage_generators.columns)

    system_dir = Path(data_folder / "pjm_energy_system_scratch")
    if system_dir.exists():
        shutil.rmtree(system_dir)
    energy_system = builder.build()
    energy_system.save(system_dir)

    #reset energy system builder
    builder = EnergySystemBuilder()
    for _, generator in class_to_test_df.iterrows():
        storage_duration = generator["Nameplate Energy Capacity (MWh)"] / generator["Nameplate Capacity (MW)"]
        if storage_duration < 4:
            storage_class = '2'
        elif 4 <= storage_duration < 6:
            storage_class = '4'
        elif 6 <= storage_duration < 8:
            storage_class = '6'
        elif 8 <= storage_duration < 10:
            storage_class = '8'
        else:
            storage_class = '12'
            
        storage_unit = StorageUnit(
            id=unit_count,
            nameplate_capacity=generator["Nameplate Capacity (MW)"],
            charge_rate=generator["Nameplate Capacity (MW)"],
            discharge_rate=generator["Nameplate Capacity (MW)"],
            charge_capacity=generator["Nameplate Energy Capacity (MWh)"],
            roundtrip_efficiency=STORAGE_EFFICIENCY,
            storage_duration = generator["Nameplate Energy Capacity (MWh)"] / generator["Nameplate Capacity (MW)"],
            storage_class = storage_class   
        )
        unit_count += 1
        builder.add_unit(storage_unit)

        # add unit to energy system
    additional_system = builder.build()

    return energy_system, additional_system 

In [5]:
class_to_test = '2'
energy_system_scratch, additional_system_scratch = load_energy_system(demand_path, start_hour, end_hour, eia_860_plants, eia_860_thermal_generators, eia_860_solar_generators, eia_860_wind_generators, eia_860_storage_generators, power_generation_file_combined, class_to_test)

In [6]:
def run_energy_system(energy_system, additional_system, start_hour_input, end_hour_input, trial_size_input):
    
    from assetra.simulation import ProbabilisticSimulation
    from assetra.metrics import ExpectedUnservedEnergy
    from assetra.contribution import EffectiveLoadCarryingCapability

    simulation = ProbabilisticSimulation(
        start_hour = start_hour_input,
        end_hour = end_hour_input,
        trial_size = trial_size_input
    )
    simulation.assign_energy_system(energy_system)
    simulation.run()

    eue_model = ExpectedUnservedEnergy(simulation)
    eue = eue_model.evaluate() / 2
   
    # initialize elcc model
    elcc_model = EffectiveLoadCarryingCapability(
        energy_system,
        ProbabilisticSimulation(
            start_hour = start_hour,
            end_hour = end_hour,
            trial_size = trial_size_input
        ),
        ExpectedUnservedEnergy
    )

    elcc = elcc_model.evaluate(additional_system)
    elcc_pct = elcc / additional_system.size * 100

    return elcc_pct

In [7]:
start_hour = "2022-01-01 00:00:00"
end_hour = "2023-12-31 23:00:00"
x = run_energy_system(energy_system_scratch, additional_system_scratch, start_hour, end_hour, 1)

In [8]:
x

87.89973958333331